In [ ]:
import math, re, os, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline

tqdm.pandas()

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# sns.set_theme(style='darkgrid')

In [ ]:
filepath = os.path.join(os.getcwd(), 'Primary Dataset', 'final_dataset.csv')
df = pd.read_csv(filepath)
df['Sale Date'] = pd.to_datetime(df['Sale Date'])

df

,Transacted Price ($),Sale Date,Area (SQM),Unit Price ($ PSM),Property Type,Postal Code,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,PPI,Police Centre,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,num_stations_1km,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW
0,"600,000",2010-01-01,95.0,6316.0,Condominium,759151,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,124.3,Yishun South NPC,33,7,705.752731,0,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"1,909,000",2010-01-02,138.0,13833.0,Condominium,98656,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,98656,124.3,Bukit Merah West NPC,58,3,1233.947139,1,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,"890,000",2010-01-03,99.0,8990.0,Condominium,597592,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,124.3,Clementi NPC,50,4,1039.586179,1,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,"700,000",2010-01-03,111.0,6306.0,Condominium,768445,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,124.3,Yishun South NPC,33,5,509.516515,1,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,"1,700,000",2010-01-04,122.0,13934.0,Condominium,98641,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,98641,124.3,Bukit Merah West NPC,58,3,1253.733260,1,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54749,"1,580,000",2020-12-31,192.0,8229.0,Condominium,798592,North East Region,Hougang,4.0,100,GERALD DRIVE,SELETAR SPRINGS CONDOMINIUM,798592,153.3,Hougang NPC,66,11,526.868822,3,464.021930,76.0,20.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54750,"870,000",2020-12-31,111.0,7838.0,Executive Condominium,545040,North East Region,Sengkang,14.0,2,RIVERVALE LINK,PARK GREEN,545040,153.3,Hougang NPC,66,15,535.286185,3,490.669867,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
54751,"1,850,000",2020-12-31,142.0,13028.0,Condominium,436904,Central Region,Kallang,14.0,120,TANJONG RHU ROAD,CASUARINA COVE,436904,153.3,Geylang NPC,133,0,2075.609094,1,600.145285,72.0,24.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
54752,"808,000",2020-12-31,58.0,13931.0,Condominium,677669,West Region,Bukit Panjang,6.0,7,DAIRY FARM HEIGHTS,THE SKYWOODS,677669,153.3,Bukit Batok NPC,51,3,397.651025,2,539.116552,91.0,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
df = pd.concat([df, pd.get_dummies(df.pop('Property Type'))], axis=1)
df

,Transacted Price ($),Sale Date,Area (SQM),Unit Price ($ PSM),Postal Code,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,PPI,Police Centre,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,num_stations_1km,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Condominium,Executive Condominium
0,"600,000",2010-01-01,95.0,6316.0,759151,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,124.3,Yishun South NPC,33,7,705.752731,0,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,"1,909,000",2010-01-02,138.0,13833.0,98656,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,98656,124.3,Bukit Merah West NPC,58,3,1233.947139,1,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,"890,000",2010-01-03,99.0,8990.0,597592,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,124.3,Clementi NPC,50,4,1039.586179,1,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,"700,000",2010-01-03,111.0,6306.0,768445,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,124.3,Yishun South NPC,33,5,509.516515,1,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
4,"1,700,000",2010-01-04,122.0,13934.0,98641,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,98641,124.3,Bukit Merah West NPC,58,3,1253.733260,1,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54749,"1,580,000",2020-12-31,192.0,8229.0,798592,North East Region,Hougang,4.0,100,GERALD DRIVE,SELETAR SPRINGS CONDOMINIUM,798592,153.3,Hougang NPC,66,11,526.868822,3,464.021930,76.0,20.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
54750,"870,000",2020-12-31,111.0,7838.0,545040,North East Region,Sengkang,14.0,2,RIVERVALE LINK,PARK GREEN,545040,153.3,Hougang NPC,66,15,535.286185,3,490.669867,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1
54751,"1,850,000",2020-12-31,142.0,13028.0,436904,Central Region,Kallang,14.0,120,TANJONG RHU ROAD,CASUARINA COVE,436904,153.3,Geylang NPC,133,0,2075.609094,1,600.145285,72.0,24.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
54752,"808,000",2020-12-31,58.0,13931.0,677669,West Region,Bukit Panjang,6.0,7,DAIRY FARM HEIGHTS,THE SKYWOODS,677669,153.3,Bukit Batok NPC,51,3,397.651025,2,539.116552,91.0,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [ ]:
print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54754 entries, 0 to 54753
Data columns (total 68 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Transacted Price ($)       54754 non-null  object        
 1   Sale Date                  54754 non-null  datetime64[ns]
 2   Area (SQM)                 54754 non-null  float64       
 3   Unit Price ($ PSM)         54754 non-null  float64       
 4   Postal Code                54754 non-null  int64         
 5   Planning Region            54754 non-null  object        
 6   Planning Area              54754 non-null  object        
 7   Floor Number               54754 non-null  float64       
 8   BLK_NO                     54754 non-null  object        
 9   ROAD_NAME                  54754 non-null  object        
 10  BUILDING                   54754 non-null  object        
 11  POSTAL                     54754 non-null  int64         
 12  PPI 

,Area (SQM),Unit Price ($ PSM),Postal Code,Floor Number,POSTAL,PPI,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,num_stations_1km,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Condominium,Executive Condominium
count,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000,54754.000000
mean,116.082222,11235.642802,478240.375826,9.935749,478240.375826,140.799693,59.868941,6.202725,677.660642,1.680188,769.355834,83.590824,11.526354,0.026500,0.110549,0.032217,0.036911,0.058644,0.021770,0.034956,0.044654,0.027304,0.036034,0.020948,0.054297,0.017569,0.038573,0.050681,0.003817,0.009077,0.000365,0.003068,0.009935,0.001059,0.004730,0.052033,0.017405,0.026044,0.011305,0.006739,0.004054,0.044599,0.024053,0.005917,0.006995,0.062790,0.005351,0.016729,0.040490,0.031833,0.096048,0.017314,0.286153,0.115809,0.159532,0.238339,0.185612,0.173887,0.695913,0.130201
std,45.122567,4469.610585,210382.467176,8.341500,210382.467176,6.948034,25.278661,3.467605,470.519186,1.828854,610.148726,7.942974,8.289676,0.160619,0.313576,0.176577,0.188544,0.234960,0.145933,0.183671,0.206546,0.162969,0.186376,0.143212,0.226606,0.131382,0.192576,0.219348,0.061665,0.094841,0.019109,0.055307,0.099181,0.032530,0.068615,0.222095,0.130777,0.159267,0.105724,0.081816,0.063546,0.206425,0.153215,0.076697,0.083343,0.242587,0.072957,0.128257,0.197108,0.175558,0.294659,0.130439,0.451966,0.319999,0.366175,0.426071,0.388797,0.379016,0.460024,0.336527
min,32.000000,3465.000000,18965.000000,1.000000,18965.000000,124.300000,28.000000,0.000000,71.016134,0.000000,41.151430,47.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,91.000000,8224.000000,348196.000000,4.000000,348196.000000,135.000000,39.000000,4.000000,379.797349,0.000000,300.586464,80.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,112.000000,10120.000000,529233.000000,8.000000,529233.000000,141.200000,58.000000,6.000000,556.301959,1.000000,593.357980,85.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [ ]:
# df['Time Period'] = df['Sale Date'].apply(lambda x: (x.year - df['Sale Date'][0].year)*12 + (x.month - df['Sale Date'][0].month))
# df

In [ ]:
df.drop(columns=['BLK_NO',
                 'BUILDING',
                 'POSTAL',
                 'Planning Area',
                 'Planning Region',
                 'Postal Code',
                 'ROAD_NAME',
                 'Sale Date',
                 'Transacted Price ($)',
                 'Police Centre',
                 'num_stations_1km',
                 #'Number of Preschools and Kindergartens',
                 'Number of Primary Schools'], inplace=True)
df

,Area (SQM),Unit Price ($ PSM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Condominium,Executive Condominium
0,95.0,6316.0,1.0,124.3,33,705.752731,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,138.0,13833.0,1.0,124.3,58,1233.947139,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,99.0,8990.0,10.0,124.3,50,1039.586179,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,111.0,6306.0,6.0,124.3,33,509.516515,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
4,122.0,13934.0,10.0,124.3,58,1253.733260,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54749,192.0,8229.0,4.0,153.3,66,526.868822,464.021930,76.0,20.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
54750,111.0,7838.0,14.0,153.3,66,535.286185,490.669867,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1
54751,142.0,13028.0,14.0,153.3,133,2075.609094,600.145285,72.0,24.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
54752,58.0,13931.0,6.0,153.3,51,397.651025,539.116552,91.0,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [ ]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df

,Area (SQM),Unit Price ($ PSM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Condominium,Executive Condominium
0,95.0,6316.0,1.0,124.3,33,705.752731,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,138.0,13833.0,1.0,124.3,58,1233.947139,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,99.0,8990.0,10.0,124.3,50,1039.586179,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,111.0,6306.0,6.0,124.3,33,509.516515,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
4,122.0,13934.0,10.0,124.3,58,1253.733260,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54669,192.0,8229.0,4.0,153.3,66,526.868822,464.021930,76.0,20.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
54670,111.0,7838.0,14.0,153.3,66,535.286185,490.669867,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1
54671,142.0,13028.0,14.0,153.3,133,2075.609094,600.145285,72.0,24.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
54672,58.0,13931.0,6.0,153.3,51,397.651025,539.116552,91.0,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [ ]:
# corr = pd.concat([df.loc[:,'Area (SQM)':'Age Sold'], df.loc[:,'BLUE':'Time Period']], axis=1).corr(method='pearson')

# plt.figure(figsize=(18, 18))
# sns.heatmap(corr, annot=True, cmap="YlGnBu",cbar_kws={'label': 'Correlation'})
# plt.tight_layout()
# plt.xticks(rotation=45, horizontalalignment='right')
# plt.yticks(rotation=0)

# plt.show()
# plt.clf()

In [ ]:
# Remove highly correlated features and drop_first features of categorical dummy features
df.drop(columns=['Age Sold',
                 'Woodlands',
                 'Condominium'], inplace=True)
df

,Area (SQM),Unit Price ($ PSM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
0,95.0,6316.0,1.0,124.3,33,705.752731,1207.822015,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,138.0,13833.0,1.0,124.3,58,1233.947139,768.529003,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,99.0,8990.0,10.0,124.3,50,1039.586179,816.818037,80.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,111.0,6306.0,6.0,124.3,33,509.516515,501.364218,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,122.0,13934.0,10.0,124.3,58,1253.733260,554.491114,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54669,192.0,8229.0,4.0,153.3,66,526.868822,464.021930,76.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
54670,111.0,7838.0,14.0,153.3,66,535.286185,490.669867,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
54671,142.0,13028.0,14.0,153.3,133,2075.609094,600.145285,72.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
54672,58.0,13931.0,6.0,153.3,51,397.651025,539.116552,91.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
# corr = pd.concat([df.loc[:,'Area (SQM)':'Remaining Lease'], df.loc[:,'BLUE':'Time Period']], axis=1).corr(method='pearson')

# plt.figure(figsize=(18, 18))
# sns.heatmap(corr, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Correlation'})
# plt.tight_layout()
# plt.xticks(rotation=45, horizontalalignment='right')
# plt.yticks(rotation=0)

# plt.show()
# plt.clf()

In [ ]:
# for feature in df.columns[:9]:
#     plt.rcParams['figure.figsize'] = 12,8
#     df[feature].hist(bins=50)
    
#     plt.title(feature)
    
#     plt.show()
#     plt.clf()

In [ ]:
# standardScale_vars = ['Area (SQM)',
#                       'Floor Number',
#                       'PPI',
#                       'Average Cases Per Year',
#                       # 'Nearest Preschool/Kindergarten',
#                       'Nearest Primary School',
#                       'nearest_station_distance',
#                       'Time Period']
# minMax_vars = ['Remaining Lease']

# s_scaler = StandardScaler()
# mm_scaler = MinMaxScaler()

# s_scaled = pd.DataFrame(s_scaler.fit_transform(df.loc[:, standardScale_vars].copy()), columns=standardScale_vars)
# mm_scaled = pd.DataFrame(mm_scaler.fit_transform(df.loc[:, minMax_vars].copy()), columns=minMax_vars)

# df = pd.concat([df.loc[:,['Unit Price ($ PSM)'].copy()], 
#                 s_scaled, 
#                 mm_scaled, 
#                 df.loc[:,'Ang Mo Kio':'Executive Condominium'].copy()], axis=1)
# df

In [ ]:
df.to_csv('modelling_dataset.csv', index=False)

## Questions
1. Is it okay to apply different types of scaling (MinMax, Standard or Robust) on different features in the same dataset?

In [ ]:
df

,Area (SQM),Unit Price ($ PSM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
0,95.0,6316.0,1.0,124.3,33,705.752731,1207.822015,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,138.0,13833.0,1.0,124.3,58,1233.947139,768.529003,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,99.0,8990.0,10.0,124.3,50,1039.586179,816.818037,80.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,111.0,6306.0,6.0,124.3,33,509.516515,501.364218,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,122.0,13934.0,10.0,124.3,58,1253.733260,554.491114,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54669,192.0,8229.0,4.0,153.3,66,526.868822,464.021930,76.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
54670,111.0,7838.0,14.0,153.3,66,535.286185,490.669867,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
54671,142.0,13028.0,14.0,153.3,133,2075.609094,600.145285,72.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
54672,58.0,13931.0,6.0,153.3,51,397.651025,539.116552,91.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
